<a href="https://colab.research.google.com/github/mdssumin/SmartFactory_with_MDSTech_2nd/blob/main/Personal_Project/Younggyu_Kim/Yolov5(dataframe).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#기본 세팅

In [ ]:
import pandas as pd
import os
import natsort
from glob import glob
import yaml
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def show_detection(path):
  image = cv2.imread(path)
  fig = plt.gcf()
  fig.set_size_inches(18,10)
  plt.axis('off')
  # cv2에서는 rgb 반전되어서 다시 돌려주어야 함
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#Yolov5 불러오기

In [ ]:
#yolov5 clone
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15814, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15814 (delta 10), reused 25 (delta 2), pack-reused 15768
Receiving objects: 100% (15814/15814), 14.64 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (10819/10819), done.


In [ ]:
#install yolov5
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.9/605.9 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


#커스텀 데이터셋 사용

###데이터셋 불러오기

In [ ]:
#roboflow에서 data 불러오기
%cd /content/
!curl -L "https://universe.roboflow.com/ds/3PKTzryPUX?key=KQX2k9moK6" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   1938      0 --:--:-- --:--:-- --:--:--  1941
100 17.7M  100 17.7M    0     0  6358k      0  0:00:02  0:00:02 --:--:-- 9190k
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/105_png_jpg.rf.abc00860d5a8609b78f4df043ece3bac.jpg  
 extracting: test/images/10_png_jpg.rf.9718a5702aaee6b430f116948890d6ec.jpg  
 extracting: test/images/11_png_jpg.rf.d2fa7f47b5b7e71c0043314cecf24ced.jpg  
 extracting: test/images/12_png_jpg.rf.1ce13b127b9e931d48c0adf41262a339.jpg  
 extracting: test/images/18_png_jpg.rf.8f2b10fec8bb68efc2e1bff6d795b4d6.jpg  
 extracting: test/images/20_png_jpg.rf.ba9a936441111993cc6721b5f9baf2fe.jpg  
 extracting: test/images

In [ ]:
#yaml 파일 확인하고 경로 맞춰주기
%cat /content/data.yaml

train: ../train/images
val: ../valid/images

nc: 3
names: ['not wheel chair', 'person', 'wheel chair']

In [ ]:
%cd /

train_img_list = glob('/content/train/images/*.jpg')
val_img_list = glob('/content/valid/images/*.jpg')

print(len(train_img_list))
print(len(val_img_list))

/
694
87


In [ ]:
with open('/content/train.txt', 'w') as t:
  t.write('\n'.join(train_img_list) + '\n')

with open('/content/val.txt', 'w') as v:
  v.write('\n'.join(val_img_list) + '\n')

In [ ]:
with open('/content/data.yaml', 'r') as f:
  data = yaml.full_load(f)
print(data)

data['train'] = '/content/train.txt'
data['val'] = '/content/val.txt'

with  open('/content/data.yaml', 'w') as f:
  yaml.dump(data, f)
print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 3, 'names': ['not wheel chair', 'person', 'wheel chair']}
{'train': '/content/train.txt', 'val': '/content/val.txt', 'nc': 3, 'names': ['not wheel chair', 'person', 'wheel chair']}


In [ ]:
%cat /content/data.yaml

names:
- not wheel chair
- person
- wheel chair
nc: 3
train: /content/train.txt
val: /content/val.txt


###학습

In [ ]:
#yolov5 학습
%cd /content/yolov5/
!python train.py --img 416 --epochs 20 --data /content/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name wheelchair_img416

/content/yolov5
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=wheelchair_img416, exist_ok=False, quad=False, c

###테스트

detect.py 수정사항  
1. save_txt, save_conf True로 변경 후, 인자로 --save-txt, --save-conf 주기  
2. 170번째줄에  
2-1. xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  
2-2. ({xywh[0]:.2f}, {xywh[1]:.2f})

In [ ]:
%cd /content/yolov5
!python detect.py --weights /content/yolov5/runs/train/wheelchair_img416/weights/best.pt --source /content/drive/MyDrive/drivedataset/wheelchair/wheelchair_ex1.jpg --save-txt --save-conf --save-crop --view-img

/content/yolov5
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

detect: weights=['/content/yolov5/runs/train/wheelchair_img416/weights/best.pt'], source=/content/drive/MyDrive/drivedataset/wheelchair/wheelchair_ex1.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False

In [ ]:
%cd /content/yolov5
!python detect.py --weights /content/yolov5/runs/train/wheelchair_img416/weights/best.pt --source /content/drive/MyDrive/drivedataset/wheelchair/wheelchair1.mp4 --save-txt --save-conf

/content/yolov5
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

detect: weights=['/content/yolov5/runs/train/wheelchair_img416/weights/best.pt'], source=/content/drive/MyDrive/drivedataset/wheelchair/wheelchair1.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False,

###데이터 프레임

In [ ]:
#txt 파일들을 하나로 합치고 column 설정
directory  = '/content/yolov5/runs/detect/exp/labels/'
outfile_name = 'merged.txt'

outfile = open(outfile_name, 'w')
outfile.write("class x y w h conf\n")

files = os.listdir(directory)
files = natsort.natsorted(files)

for filename in files:
  if '.txt' not in filename:
    continue
  file = open(directory + filename)
  for line in file:
    outfile.write(line)
  file.close()
outfile.close()

In [ ]:
#txt to dataframe
%cd /content/yolov5/

df = pd.read_csv('merged.txt', sep = ' ')

/content/yolov5


In [ ]:
df.head()

,class,x,y,w,h,conf
0,2,0.516276,0.638657,0.088802,0.181944,0.856627
1,1,0.517187,0.556713,0.101562,0.264352,0.881082
2,2,0.516406,0.640972,0.088542,0.178241,0.861500
3,1,0.517187,0.559954,0.100000,0.280093,0.881775
4,2,0.516536,0.641204,0.087240,0.178704,0.862202
